In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

import time
from key import id, password

prefs = {
        "download.default_directory": "C:\\Users\\MIRAI\\programs\\syllabus\\pdf\\",
        "download.prompt_for_download": False,
        "plugins.always_open_pdf_externally": True,
        }

ID = id
PASS = password

In [ ]:
options = Options()

options.add_experimental_option("detach", True)

options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=options)

driver.get("https://webcit-5.edu.metro-cit.ac.jp/users/sign_in")

driver.find_element(By.NAME, "user[name]").send_keys(ID)
driver.find_element(By.NAME, "user[password]").send_keys(PASS)

driver.find_element(By.NAME, "commit").click()

In [ ]:
time.sleep(1)

driver.execute_script("window.open('https://webcit-5.edu.metro-cit.ac.jp/gnumbers/33494/last_hyoukas');")

# 現在のウィンドウハンドルを取得
original_window = driver.current_window_handle

# 新しいタブまたはウィンドウに切り替え
new_window = [window for window in driver.window_handles if window != original_window][0]
driver.switch_to.window(new_window)

In [ ]:
rows = driver.find_elements(By.CSS_SELECTOR, "table.bordered tr")

for row in rows:
    cells = row.find_elements(By.CSS_SELECTOR, "td")
    if cells:
        last_cell = cells[-1]
        if (last_cell.text != "PDFファイルが作成させていません。"):
            # print(last_cell.text)
            last_cell.click()

In [ ]:
import os
from PyPDF2 import PdfReader, PdfWriter

def merge_pdfs(directory, output_filename):
    # PdfWriter オブジェクトの初期化
    writer = PdfWriter()

    # 指定されたディレクトリ内のファイルをループする
    for item in sorted(os.listdir(directory)):
        if item.endswith('.pdf'):
            # PDFファイルを開く
            filepath = os.path.join(directory, item)
            reader = PdfReader(filepath)

            # 各ページを書き込む
            for page in reader.pages:
                writer.add_page(page)

    # 結合したPDFを保存
    with open(output_filename, 'wb') as f:
        writer.write(f)

# 使用例
directory = './pdf'
output_filename = './output.pdf'
merge_pdfs(directory, output_filename)

# sbiにあるか確認

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import time
import json
import requests
from bs4 import BeautifulSoup
from key import sbi_id, sbi_pass

def remove_symbol_from_json(symbol):
    with open('./scrape_data/stock_name.json', 'r', encoding="utf-8") as file:
        data = json.load(file)

    for category in data:
        data[category] = [company for company in data[category] if company["symbol"] != symbol]

    with open('./scrape_data/stock_name.json', 'w', encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

caps = DesiredCapabilities.CHROME
caps['goog:loggingPrefs'] = {'performance': 'ALL'}

options = Options()
options.add_experimental_option("detach", True)
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, Gecko) Chrome/58.0.3029.110 Safari/537.36")

driver = webdriver.Chrome(options=options)

driver.get("https://www.sbisec.co.jp/ETGate")

id_input = driver.find_element(By.NAME, "user_id")
id_input.send_keys(sbi_id)

pass_input = driver.find_element(By.NAME, "user_password")
pass_input.send_keys(sbi_pass)

driver.find_element(By.NAME, "ACT_login").click()

print("Please input Enter")
input("")

driver.get("https://global.sbisec.co.jp/home")
time.sleep(0.3)

driver.find_element(By.XPATH, r'//*[@id="root"]/main/article/div[1]/div[2]/a[2]').click()

# JSONファイルの読み込み
with open('./scrape_data/stock_name.json', 'r', encoding="utf-8") as file:
    data = json.load(file)

# 全てのシンボルをプリント
for category in data:
    for company in data[category]:
        stock_code = company["symbol"]
        stock_name = company["company_name"]
        print(stock_code, ':', stock_name)

        try:
            driver.get(f"https://global.sbisec.co.jp/invest/us/stock/{stock_code}?resource=news&searchType=include")
            time.sleep(0.3)

            iframe = driver.page_source
            soup = BeautifulSoup(iframe, "html.parser")
            url_pattern = "https://graph.sbisec.co.jp/sbinews/pc?"
            iframe_urls = [iframe.get('src') for iframe in soup.find_all('iframe', src=True) if iframe['src'].startswith(url_pattern)]

            token = iframe_urls[0].split("token=")[1]
            url_head = f"https://graph.sbisec.co.jp/sbinews/srvdetail?symbol={stock_code}&token={token}"
            print(url_head)

            res_head = requests.get(url_head)
            data_head = res_head.json()

            len_head = len(data_head['data'])
            time.sleep(1)

        except Exception as e:
            print(f"Error processing {stock_code}: {e}")
            remove_symbol_from_json(stock_code)
            print(f"Removed {stock_code} from stock_name.json")


In [ ]:
import requests
import json

# stockData = json.load(open("./scrape_data/stock_name.json", encoding="cp932"))

with open('./stock_name.json', 'r', encoding="utf-8") as file:
    stockData = json.load(file)

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100",
    "Accept-Language": "ja,en;q=0.9,en-GB;q=0.8,en-US;q=0.7",
}

for category in stockData:
    for company in stockData[category]:
        stock_code = company["symbol"]
        response = requests.get(
            f"https://api.nasdaq.com/api/quote/{stock_code}/info?assetclass=stocks",
            headers=headers,
        )
        fetchedData = response.json()
        if fetchedData["data"] == None:
            print("アッ！！", stock_code)
            # break

        print("ok", stock_code)

In [ ]:
url = 'https://api.nasdaq.com/api/quote/NVDA/historical?assetclass=stocks&fromdate=2024-01-01&limit=9999&todate=2024-07-27&random=43'
res = requests.get(url, headers=headers)

data = res.json()
print(data['data']['tradesTable']['rows'])

date_close_list = [{"date": row["date"], "close": row["close"]} for row in data["data"]["tradesTable"]["rows"]]

dates = [row["date"] for row in data["data"]["tradesTable"]["rows"]]
closes = [row["close"] for row in data["data"]["tradesTable"]["rows"]]

print(date_close_list[0]['date'])
print("Dates:", dates)
print("Closes:", closes)

In [ ]:
import requests
import json
import pandas as pd

BASE_URL = 'http://192.168.1.222:8999'

with open('./stock_name.json', 'r', encoding='utf-8') as file:
    stockData = json.load(file)

for category in stockData:
    for company in stockData[category]:
        stock_code = company['symbol']
        url = f'{BASE_URL}/getdays/{stock_code}'
        res = requests.get(url)
        print(res)
        df = pd.DataFrame(res.json())
        df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
        date_time_array = df['datetime'].to_numpy()
        print(df.info())
        # print(date_time_array)

# values get

In [ ]:
import requests
import json
from datetime import datetime

stocks_data = {}
today = datetime.today().strftime('%Y-%m-%d')

with open('./stock_name.json', 'r', encoding="utf-8") as file:
    stockData = json.load(file)

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100",
    "Accept-Language": "ja,en;q=0.9,en-GB;q=0.8,en-US;q=0.7",
}

for category in stockData:
    for company in stockData[category]:
        stock_code = company["symbol"]

        url = f'https://api.nasdaq.com/api/quote/{stock_code}/historical?assetclass=stocks&fromdate=2024-01-01&limit=9999&todate={today}'

        response = requests.get(url, headers=headers)

        print(stock_code)
        print(response)

        if response.status_code == 200:
            # レスポンスから必要なデータを抽出（例：response.json()から 'rows' 部分を抽出）
            stock_data = response.json()["data"]["tradesTable"]["rows"]
            # 必要なフィールドのみ抽出して辞書に追加
            stocks_data[stock_code] = [{"date": row["date"], "close": row["close"]} for row in stock_data]
        else:
            print(f"Failed to retrieve data for {stock_code}")

with open('./stocks_values.json', 'w') as json_file:
    json.dump(stocks_data, json_file, indent=4)

print("JSONファイルに保存されました。")

In [ ]:
import requests
import json
from key import BASE_URL

url = f'{BASE_URL}/null_values/ZTS'
res = requests.get(url)
data = res.json()

for date, id_list in data.items():
    print(date)
    # for id in id_list:
        # print(id)

In [ ]:
import requests
import json
from datetime import datetime, timedelta

BASE_URL = 'http://192.168.1.222:8999'
post_url = f'{BASE_URL}/spot_update'

def find_previous_value(stock_symbol, date_str, json_days, entries):
    # 再帰的に1日ずつ前にさかのぼる
    date_obj = datetime.strptime(date_str, "%Y-%m-%d") - timedelta(days=1)
    previous_date_str = date_obj.strftime("%Y-%m-%d")

    # エントリーに前日の日付が存在するか確認
    for entry in entries:
        if entry['date'] == previous_date_str:
            return entry['close']  # 存在すればその値を返す

    # 存在しなければさらに1日さかのぼる
    return find_previous_value(stock_symbol, previous_date_str, json_days, entries)

def remove_symbol_from_json(symbol):
    with open('./stock_name.json', 'r', encoding="utf-8") as file:
        data = json.load(file)

    for category in data:
        data[category] = [company for company in data[category] if company["symbol"] != symbol]

    with open('./stock_name.json', 'w', encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

with open('./stocks_values.json', 'r') as file:
    data = json.load(file)

for stock_symbol, entries in reversed(data.items()):
    print(f'Stock Symbol: {stock_symbol}')

    days_url = f'{BASE_URL}/null_values/{stock_symbol}'
    days = requests.get(days_url).json()

    json_days = list(set(entry['date'] for entry in entries))

    try:
        for date, id_list in days.items():
            if date not in json_days:
                update_value = find_previous_value(stock_symbol, date, json_days, entries)
                print(f'Missing date: {date}, filling with previous value: {update_value}')

                for id in id_list:
                    data = {
                        'id' : id,
                        'column_name' : 'value',
                        'update_value' : update_value
                    }
                    # print(data)

                    response = requests.post(post_url, json=data)
                    print(response)
            else:
                for entry in entries:
                    if entry['date'] == date:
                        update_value = entry['close']
                        print(f"Date: {date}, Close: {update_value}")

                        for id in id_list:
                            data = {
                                'id' : id,
                                'column_name' : 'value',
                                'update_value' : update_value
                            }
                            # print(data)

                            response = requests.post(post_url, json=data)
                            print(response)

    except Exception as e:
        print(f'Error: {e}')
        remove_symbol_from_json(stock_symbol)

: 

In [ ]:
import requests
import json
from datetime import datetime, timedelta

BASE_URL = 'http://192.168.1.222:8999'
post_url = f'{BASE_URL}/spot_update'
today = datetime.today().strftime('%Y-%m-%d')

values_data = {}

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100",
    "Accept-Language": "ja,en;q=0.9,en-GB;q=0.8,en-US;q=0.7",
}

with open('./stock_name.json', 'r', encoding="utf-8") as file:
    stockData = json.load(file)

def convert_date(date_str):
    date_obj = datetime.strptime(date_str, '%m/%d/%Y')

    conved_date = date_obj.strftime('%Y-%m-%d')

    return conved_date

def remove_symbol_from_json(symbol):
    with open('./stock_name.json', 'r', encoding="utf-8") as file:
        data = json.load(file)

    for category in data:
        data[category] = [company for company in data[category] if company["symbol"] != symbol]

    with open('./stock_name.json', 'w', encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

for category in stockData:
    for company in stockData[category]:
        stock_code = company["symbol"]

        url = f'https://api.nasdaq.com/api/quote/{stock_code}/historical?assetclass=stocks&fromdate=2024-01-01&limit=9999&todate={today}'

        response = requests.get(url, headers=headers)

        print(stock_code)
        print(response)
        try:
            if response.status_code == 200:
                # レスポンスから必要なデータを抽出（例：response.json()から 'rows' 部分を抽出）
                stock_data = response.json()["data"]["tradesTable"]["rows"]
                print(stock_data)
                # 必要なフィールドのみ抽出して辞書に追加
                values_data[stock_code] = [{"date": convert_date(row["date"]), "close": row["close"].replace('$', '')} for row in stock_data]
            else:
                print(f"Failed to retrieve data for {stock_code}")

        except Exception as e:
            print(e)
            remove_symbol_from_json(stock_code)
            continue

with open('./stocks_values.json', 'w') as json_file:
    json.dump(values_data, json_file, indent=4)

print("saved stocks_values.json")
